# Settings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install wandb -qqq
!wandb login

import os
os.environ["WANDB_ENTITY"] = "capi-bgu"
os.environ['WANDB_CONSOLE'] = "off"
os.environ['WANDB_SILENT'] = "true"

In [ ]:
!pip install git+https://github.com/capi-bgu/oratio.git -qqq

In [ ]:
!git clone https://github.com/capi-bgu/Research.git

In [ ]:
import numpy as np
import pickle
import cv2

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras

from Research.util.data_loading import *
from Research.util.evaluate import *
from Research.models.emotionnet_nano import *
from Research.models.frame_attention import *
from oratio.Session import Session

# Data Loading

In [ ]:
def create_dataset(dataframe, sequence_length=None, resize_to=None, labels=("categorical"), test_split=0.1, val_split=None, random_state=None, category_encoder=None):
  if category_encoder is None:
    category_encoder = LabelEncoder()
    category_encoder.fit(dataframe["categorical"])
  dataframe["categorical"] = category_encoder.transform(dataframe["categorical"])
  raw_labels = dataframe[labels].values.astype(np.int)
  raw_data = dataframe["images"].values

  for i, images in enumerate(raw_data):
    if resize_to is not None:
      images = np.array(images)
      resized_images = np.zeros(shape=((len(images), ) + resize_to))
      for j, image in enumerate(images):  
        resized_images[j] = cv2.resize(image, resize_to)
      raw_data[i] = resized_images
    else:
      raw_data[i] = np.array(images)
  
  train_data, test_data, train_labels, test_labels = train_test_split(raw_data,
                                                                      raw_labels,
                                                                      test_size=test_split,
                                                                      random_state=random_state)
  if val_split is not None:
    train_data, val_data, train_labels, val_labels = train_test_split(train_data,
                                                                      train_labels,
                                                                      test_size=val_split,
                                                                      random_state=random_state)

  if sequence_length is not None:
    unwraped_train_data = generate_sequences(train_data[0], sequence_length)
    unwraped_test_data = generate_sequences(test_data[0], sequence_length)
    if val_split is not None:
      unwraped_val_data = generate_sequences(val_data[0], sequence_length)
  else:
    unwraped_train_data = train_data[0]
    unwraped_test_data = test_data[0]
    if val_split is not None:
      unwraped_val_data = test_data[0]

  unwraped_train_labels = np.repeat(np.expand_dims(train_labels[0], axis=0), len(unwraped_train_data), axis=0)
  unwraped_test_labels = np.repeat(np.expand_dims(test_labels[0], axis=0), len(unwraped_test_data), axis=0)
  if val_split is not None:
    unwraped_val_labels = np.repeat(np.expand_dims(val_labels[0], axis=0), len(unwraped_val_data), axis=0)

  
  for i in range(1, len(train_data)):
    if sequence_length is not None:
      train_examples = generate_sequences(train_data[i], sequence_length)
      if train_examples is None:
        continue
    else:
      train_examples = train_data[i]
    unwraped_train_data = np.concatenate((unwraped_train_data, train_examples))
    unwraped_train_labels = np.concatenate(
        (
          unwraped_train_labels,
          np.repeat(np.expand_dims(train_labels[i], axis=0), len(train_examples), axis=0)
        )
    )

  for i in range(1, len(test_data)):
    if sequence_length is not None:
      test_examples = generate_sequences(test_data[i], sequence_length)
      if test_examples is None:
        continue
    else:
      test_examples = test_data[i]
    unwraped_test_data = np.concatenate((unwraped_test_data, test_examples))
    unwraped_test_labels = np.concatenate(
      (
        unwraped_test_labels,
        np.repeat(np.expand_dims(test_labels[i], axis=0), len(test_examples), axis=0)
      )
    )
  
  if val_split is not None:
    for i in range(1, len(val_data)):
      if sequence_length is not None:
        val_examples = generate_sequences(val_data[i], sequence_length)
        if val_examples is None:
          continue
      else:
        val_examples = val_data[i]
      unwraped_val_data = np.concatenate((unwraped_val_data, val_examples))
      unwraped_val_labels = np.concatenate(
        (
          unwraped_val_labels,
          np.repeat(np.expand_dims(val_labels[i], axis=0), len(val_examples), axis=0)
        )
      )
      
  res = (unwraped_train_data / 255, unwraped_test_data / 255)
  if val_split is not None:
    res += (unwraped_val_data / 255, )
  
  res += (unwraped_train_labels, unwraped_test_labels)
  if val_split is not None:
    res += (unwraped_val_labels, )

  if "categorical" in labels:
    return res, category_encoder
  
  return res

def generate_sequences(arr, sequence_length):
  if len(arr) < sequence_length:
    return None
  sequences = np.zeros(shape=(len(arr) - (sequence_length - 1), sequence_length) + arr.shape[1:])
  for i in range(len(arr) - (sequence_length - 1)):
    sequences[i] = arr[i:i+sequence_length]
  
  return sequences

# EmotionNet Nano 

In [ ]:
!nvidia-smi

In [ ]:
import gc

def use_bin(model):
  last_layer = model.layers[-2].output
  pred = keras.layers.Dense(2, activation="softmax")(last_layer)
  return keras.Model(inputs=model.inputs, outputs=pred)

def use_reg(model):
  last_layer = model.layers[-2].output
  pred = keras.layers.Dense(3)(last_layer)
  return keras.Model(inputs=model.inputs, outputs=pred)

class ENNEvaluator(Evaluator):

  def __init__(self, project, name, task="categorical", use_logger=False,
               use_ensemble=True, model_path: str = None, scorer: Scorer = None):
    
    self.loss = "sparse_categorical_crossentropy"
    self.metrics = ["accuracy"]


    if task == "categorical":
      super(ENNEvaluator, self).__init__(project, name, use_logger=use_logger,
                                         model_path=model_path, scorer=scorer)
      self.model_pred = lambda x: x

    elif task == "positive":
      super(ENNEvaluator, self).__init__(project, name, use_logger=use_logger,
                                         model_path=model_path, scorer=scorer)
      self.model_pred = use_bin

    elif task == "regression":
      super(ENNEvaluator, self).__init__(project, name, use_logger=use_logger,
                                         task=Task.REG, model_path=model_path, scorer=scorer)
      self.model_pred = use_reg
      self.loss = "mse"
      self.metrics = ["mae"]

  def model_config(self):
    return {
        "optimizer": 'adam',
        "loss": self.loss,
        "metrics": self.metrics,
        "batch_size": 100,
        "epochs": 100,
    }

  def train(self, train_data, validation_data, callbacks):
    config = self.model_config()
    model = emotion_nano_b()
    model = self.model_pred(model)
    model.compile(optimizer=config["optimizer"], loss=config["loss"], metrics=config["metrics"])
    model.fit(train_data[0], train_data[1], validation_data=validation_data,
              batch_size=config["batch_size"], epochs=config["epochs"],
              callbacks=callbacks)

    gc.collect()
    return model

  def predict(self, model, data):
    return model(data)

In [ ]:
import traceback
import gc

for name in ("ron", "yuval", "tal", "liraz", "shiran", "tomer", "liraz", "niv"):
  for duration in (1, 5, 10):
    features = load_features(f"/content/drive/MyDrive/capi/data/public/{name}.db", "Camera", duration)

    try:
      base_path = "/content/drive/MyDrive/capi/models/frame_attention"
      
      # categorical
      gc.collect()
      (train_data, test_data, train_labels, test_labels), category_encoder = create_dataset(features, resize_to=(48, 48), test_split=0.2, random_state=42)
      gc.collect()
      label_map = {k: v for k, v in zip(category_encoder.transform(category_encoder.classes_), category_encoder.classes_)}
      cat_eval = ENNEvaluator(project="capi", name=f"emotionnet-nano_cat_{name}-{duration}d",
                              use_logger=True, model_path=f"{base_path}/emotionnet-nano_cat-{name}-{duration}/")
      cat_eval.evaluate((train_data, train_labels), (test_data, test_labels), label_map, splits=10)

      # positive
      del train_data, test_data, train_labels, test_labels, cat_eval
      gc.collect()
      (train_data, test_data, train_labels, test_labels) = create_dataset(features, resize_to=(48, 48),
                                                                          test_split=0.2, random_state=42,
                                                                          labels="positive")
      gc.collect()
      label_map = {0: "negative", 1: "positive"}
      pos_eval = ENNEvaluator(project="capi", name=f"emotionnet-nano_pos_{name}-{duration}d",
                              use_logger=True, model_path=f"{base_path}/emotionnet-nano_pos-{name}-{duration}/",
                              task="positive")
      pos_eval.evaluate((train_data, train_labels), (test_data, test_labels), label_map, splits=10)

      # regression
      del train_data, test_data, train_labels, test_labels, pos_eval
      gc.collect()
      (train_data, test_data, train_labels, test_labels) = create_dataset(features, resize_to=(48, 48),
                                                                          test_split=0.2, random_state=42,
                                                                          labels=["valance", "arousal", "dominance"])
      reg_eval = ENNEvaluator(project="capi", name=f"emotionnet-nano_reg_{name}-{duration}d",
                              use_logger=True, model_path=f"{base_path}/emotionnet-nano_reg-{name}-{duration}/",
                              task="regression")
      reg_eval.evaluate((train_data, train_labels), (test_data, test_labels), splits=10)
      del train_data, test_data, train_labels, test_labels, reg_eval
      gc.collect()

    except Exception as e:
      print(f"-----------------ERROR-{name}-----------------")
      print(e)
      print(traceback.print_exception(type(e), e, e.__traceback__))
      print(f"---------------END-ERROR-{name}---------------")


In [ ]:
all = ("ron", "yuval", "liraz", "shiran", "tomer", "niv", "tal")
base_path = "/content/drive/MyDrive/capi/models/frame_attention"
duration = 10

train_names, (test_name, ) = train_test_split(all, test_size=0.1, random_state=42)
train_features = [load_features(f"/content/drive/MyDrive/capi/data/public/{name}.db", "Camera", duration) for name in train_names]
test_features = load_features(f"/content/drive/MyDrive/capi/data/public/{test_name}.db", "Camera", duration)
(_, test_data, _, test_labels), category_encoder = create_dataset(test_features, resize_to=(48, 48), test_split=0.99, random_state=42)

(train_data, curr_test_data, train_labels, curr_test_labels), category_encoder = create_dataset(train_features[0], resize_to=(48, 48),
                                                                                                test_split=0.05, random_state=42,
                                                                                                category_encoder=category_encoder)

test_data = np.concatenate((test_data, curr_test_data), axis=0)
test_labels = np.concatenate((test_labels, curr_test_labels), axis=0)

for i in range(1, len(train_features)):
  testee_features = train_features[i]
  (curr_train_data, curr_test_data, curr_train_labels, curr_test_labels), category_encoder = create_dataset(testee_features, resize_to=(48, 48),
                                                                                          test_split=0.05, random_state=42,
                                                                                          category_encoder=category_encoder)

  train_data = np.concatenate((train_data, curr_train_data), axis=0)
  train_labels = np.concatenate((train_labels, curr_train_labels), axis=0)
  test_data = np.concatenate((test_data, curr_test_data), axis=0)
  test_labels = np.concatenate((test_labels, curr_test_labels), axis=0)

In [ ]:
cat_eval = ENNEvaluator(project="capi", name=f"emotionnet-nano_cat_general_{duration}d",
                              use_logger=True, model_path=f"{base_path}/emotionnet-nano_cat-general-{duration}/")
label_map = {k: v for k, v in zip(category_encoder.transform(category_encoder.classes_), category_encoder.classes_)}
cat_eval.evaluate((train_data, train_labels), (test_data, test_labels), label_map, splits=10)

# Frame Attention Model

In [ ]:
camera_features = load_features("/content/drive/MyDrive/capi/data/public/ron.db", "Camera", 10)

In [ ]:
(train_data, test_data, val_data, train_labels, test_labels, val_labels), category_encoder = create_dataset(camera_features, sequence_length=5, resize_to=(48, 48), test_split=0.2)

In [ ]:
frame_attention_model = fanet()
frame_attention_model.compile(optimizer=keras.optimizers.Adam(0.1), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

def lr_schedular(epoch, lr):
  if epoch == 10:
    return lr * 1e-1
  if epoch == 20:
    return lr * 1e-2
  if epoch == 30:
    return lr * 1e-3

  return lr

callbacks = [
  keras.callbacks.ModelCheckpoint("frame_attention/", monitor='val_accuracy', save_best_only=True, save_weights_only=True),
  keras.callbacks.LearningRateScheduler(lr_schedular)
]
frame_attention_model.fit(np.expand_dims(train_data, axis=4), train_labels, validation_data=(val_data, val_labels), batch_size=64, epochs=60, callbacks=callbacks)
frame_attention_model.save_weights("frame_attention_last/")

In [ ]:
frame_attention_model.load_weights("frame_attention_last/")
frame_attention_model.evaluate(np.expand_dims(test_data, axis=4), test_labels, batch_size=10)
frame_attention_model.evaluate(np.expand_dims(val_data, axis=4), val_labels, batch_size=10)

In [ ]:
frame_attention_model.load_weights("frame_attention/")
frame_attention_model.evaluate(np.expand_dims(test_data, axis=4), test_labels, batch_size=10)
frame_attention_model.evaluate(np.expand_dims(val_data, axis=4), val_labels, batch_size=10)